<h1 align=center > Clustering Cities In INDIA <h1>

### Introduction 
   
   India is the second largest country in the world after China .There are 4000 cities and towns in India .About 300 cities have population over 1,00,000  Seven cities have population more than 3 million .Greater  Mumbai still is the most populated city in its 440sq.Km. area followed by Delhi, Kolkata ,Bangalore and Chennai. 
In this project over 150 cites data are collected from different web sources and use it for clustering cities in India. These cities are clustered on basis of the venues in the city. By this we can show same kind of cities on basis venues in it. 

### Importing Required Libraries 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install  folium 
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 6.1MB/s ta 0:00:011
Libraries imported.


### Download and Explore Dataset

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_cities_in_India_by_population"
cities_df = pd.read_html(url)[0]
cities_df.head()
    

,Rank,City,Population(2011)[3],Population(2001),State or union territory
0,1,Mumbai,12442373,11978450,Maharashtra
1,2,Delhi,11007835,9879172,Delhi
2,3,Bangalore,8436675,4301326,Karnataka
3,4,Hyderabad,6809970,3637483,Telangana
4,5,Ahmedabad,5570585,3520085,Gujarat


In [5]:
cities_df = cities_df.drop(['Rank','Population(2011)[3]','Population(2001)'],axis = 1)
cities_df = cities_df.rename(columns = {"Population(2011)[3]" : "p2011","Population(2001)" : "p2001"})
cities_df.head()

,City,State or union territory
0,Mumbai,Maharashtra
1,Delhi,Delhi
2,Bangalore,Karnataka
3,Hyderabad,Telangana
4,Ahmedabad,Gujarat


In [6]:
cities_df['City'] = cities_df['City'].apply(lambda x : x.split('[')[0])

In [7]:
#using geoloccator obtaining longitude and latitude values of each city
longitude = []
latitude = []
for address in cities_df['City']:
    address = address+',India'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(np.nan)
        longitude.append(np.nan)
    print(address, end=" ")
    
print('done')

Mumbai,India Delhi,India Bangalore,India Hyderabad,India Ahmedabad,India Chennai,India Kolkata,India Surat,India Pune,India Jaipur,India Kanpur,India Nagpur,India Lucknow,India Visakhapatnam,India Thane,India Bhopal,India Indore,India Pimpri-Chinchwad,India Patna,India Vadodara,India Ghaziabad,India Ludhiana,India Agra,India Nashik,India Faridabad,India Meerut,India Rajkot,India Kalyan-Dombivli,India Vasai-Virar,India Varanasi,India Srinagar,India Aurangabad,India Dhanbad,India Amritsar,India Navi Mumbai,India Allahabad,India Howrah,India Ranchi,India Gwalior,India Jabalpur,India Coimbatore,India Vijayawada,India Jodhpur,India Madurai,India Raipur,India Chandigarh,India Guwahati,India Solapur,India Hubli–Dharwad,India Mysore,India Tiruchirappalli,India Bareilly,India Aligarh,India Tiruppur,India Gurgaon,India Moradabad,India Jalandhar,India Bhubaneswar,India Salem,India Warangal,India Mira-Bhayandar,India Jalgaon,India Kota,India Guntur,India Bhiwandi,India Saharanpur,India Gorakhpur,I

In [8]:
cities_df['longitude'] = longitude
cities_df['latitude'] = latitude
cities_df.head()

,City,State or union territory,longitude,latitude
0,Mumbai,Maharashtra,72.835335,18.938771
1,Delhi,Delhi,77.221939,28.651718
2,Bangalore,Karnataka,77.591300,12.979120
3,Hyderabad,Telangana,78.461065,17.388786
4,Ahmedabad,Gujarat,72.579707,23.021624


In [9]:
cities_df.isnull().sum()

City                        0
State or union territory    0
longitude                   5
latitude                    5
dtype: int64

In [10]:
#removing cities with null values 
df = cities_df.dropna(axis = 0)
df.head()

,City,State or union territory,longitude,latitude
0,Mumbai,Maharashtra,72.835335,18.938771
1,Delhi,Delhi,77.221939,28.651718
2,Bangalore,Karnataka,77.591300,12.979120
3,Hyderabad,Telangana,78.461065,17.388786
4,Ahmedabad,Gujarat,72.579707,23.021624


In [11]:
df.isnull().sum()

City                        0
State or union territory    0
longitude                   0
latitude                    0
dtype: int64

In [12]:
# getting longitude and latitude data of India
address = 'India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of India are 22.3511148, 78.6677428.


In [13]:
#plotting map of India with cities marked on it.
map_india = folium.Map(location=[latitude,longitude],zoom_start = 5)

for lat, lng, city in zip(df['latitude'], df['longitude'],df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_india)  
    
map_india

### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = '115HWRNCE1UQJRQZ1RRONOFWUWSG1SRN0UPETDWD3EMNHH2K' # your Foursquare ID
CLIENT_SECRET = 'ZLDIGK2HJD0K2T4NPMRMYZRT2P4Y1XMIYSZ0VR4MAHWA4F2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 115HWRNCE1UQJRQZ1RRONOFWUWSG1SRN0UPETDWD3EMNHH2K
CLIENT_SECRET:ZLDIGK2HJD0K2T4NPMRMYZRT2P4Y1XMIYSZ0VR4MAHWA4F2J


In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
#fuction to get venues in each city.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
city_venues = getNearbyVenues(names=df['City'],latitudes=df['latitude'],longitudes=df['longitude'])

Mumbai
Delhi
Bangalore
Hyderabad
Ahmedabad
Chennai
Kolkata
Surat
Pune
Jaipur
Kanpur
Nagpur
Lucknow
Visakhapatnam
Thane
Bhopal
Indore
Pimpri-Chinchwad
Patna
Vadodara
Ghaziabad
Ludhiana
Agra
Nashik
Faridabad
Meerut
Rajkot
Kalyan-Dombivli
Vasai-Virar
Varanasi
Srinagar
Aurangabad
Dhanbad
Amritsar
Navi Mumbai
Allahabad
Howrah
Ranchi
Gwalior
Jabalpur
Coimbatore
Vijayawada
Jodhpur
Madurai
Raipur
Chandigarh
Guwahati
Solapur
Hubli–Dharwad
Mysore
Tiruchirappalli
Bareilly
Aligarh
Tiruppur
Gurgaon
Moradabad
Jalandhar
Bhubaneswar
Salem
Warangal
Mira-Bhayandar
Jalgaon
Kota
Guntur
Bhiwandi
Saharanpur
Gorakhpur
Bikaner
Amravati
Noida
Jamshedpur
Bhilai
Cuttack
Firozabad
Kochi
Nellore
Bhavnagar
Dehradun
Durgapur
Asansol
Rourkela
Nanded
Kolhapur
Ajmer
Akola
Gulbarga
Jamnagar
Ujjain
Loni
Siliguri
Jhansi
Ulhasnagar
Jammu
Sangli-Miraj & Kupwad
Mangalore
Erode
Belgaum
Ambattur
Tirunelveli
Malegaon
Gaya
Thiruvananthapuram
Udaipur
Kakinada
Davanagere
Kozhikode
Maheshtala
Rajpur Sonarpur
Rajahmundry
Bokaro
Sout

In [19]:
city_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mumbai,18.938771,72.835335,Town House Cafe,18.938550,72.833464,Bar
1,Mumbai,18.938771,72.835335,Royal China,18.938715,72.832933,Chinese Restaurant
2,Mumbai,18.938771,72.835335,Sher-E-Punjab,18.937944,72.837853,Indian Restaurant
3,Mumbai,18.938771,72.835335,Cafe Excelsior,18.937701,72.833566,Café
4,Mumbai,18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train Station


In [20]:
city_data = pd.get_dummies(city_venues[['Venue Category']],prefix="",prefix_sep="")
city_data['City'] = city_venues['City']
city_data.head()

,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brewery,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Café,Camera Store,Campground,Candy Store,Capitol Building,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,City,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Grocery Store,Gujarati Restaurant,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nightclub,Optical Shop,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Stadium,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [21]:
city_grouped = city_data.groupby('City').mean().reset_index()
city_grouped.head()

,City,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brewery,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Café,Camera Store,Campground,Candy Store,Capitol Building,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Grocery Store,Gujarati Restaurant,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nightclub,Optical Shop,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Stadium,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Wine Shop,Women's Store
0,Agartala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,Agra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,Ahmedabad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [22]:
city_grouped.shape

(152, 155)

## Cluster Cities

In [23]:
# set number of clusters
kclusters = 5

city_grouped_clustering = city_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 3, 3, 0, 3, 3, 3, 4, 2], dtype=int32)

In [28]:
l = list(kmeans.labels_)
ndf = pd.DataFrame()
ndf['Cluster label'] = l
ndf['City'] = city_grouped['City']
ndf.head()

,Cluster label,City
0,3,Agartala
1,4,Agra
2,3,Ahmedabad
3,3,Aizawl
4,0,Ajmer


In [29]:
# adding cluster label to the df
final_df = pd.merge(ndf,df,on = 'City')
final_df.head()

,Cluster label,City,State or union territory,longitude,latitude
0,3,Agartala,Tripura,91.282382,23.831238
1,4,Agra,Uttar Pradesh,78.009816,27.175255
2,3,Ahmedabad,Gujarat,72.579707,23.021624
3,3,Aizawl,Mizoram,92.720930,23.741409
4,0,Ajmer,Rajasthan,74.639000,26.469100


In [31]:
final_df.shape

(153, 5)

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['latitude'],final_df['longitude'],final_df['City'],final_df['Cluster label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  top 10 venues in each city

In [51]:
#let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
#the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = city_grouped['City']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agartala,Salad Place,Fast Food Restaurant,Multiplex,Mobile Phone Shop,Indian Restaurant,Women's Store,Department Store,Discount Store,Diner,Dhaba
1,Agra,Clothing Store,Camera Store,Hotel,Indian Restaurant,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
2,Ahmedabad,Park,Art Gallery,River,Castle,Clothing Store,Dessert Shop,Dumpling Restaurant,Donut Shop,Discount Store,Chettinad Restaurant
3,Aizawl,Restaurant,Mobile Phone Shop,Women's Store,Department Store,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop,Dance Studio
4,Ajmer,Indian Restaurant,Clothing Store,Vegetarian / Vegan Restaurant,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba


In [55]:
final_df = pd.merge(final_df,city_venues_sorted,on = 'City')
final_df.head()

,Cluster label,City,State or union territory,longitude,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Agartala,Tripura,91.282382,23.831238,Salad Place,Fast Food Restaurant,Multiplex,Mobile Phone Shop,Indian Restaurant,Women's Store,Department Store,Discount Store,Diner,Dhaba
1,4,Agra,Uttar Pradesh,78.009816,27.175255,Clothing Store,Camera Store,Hotel,Indian Restaurant,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner
2,3,Ahmedabad,Gujarat,72.579707,23.021624,Park,Art Gallery,River,Castle,Clothing Store,Dessert Shop,Dumpling Restaurant,Donut Shop,Discount Store,Chettinad Restaurant
3,3,Aizawl,Mizoram,92.720930,23.741409,Restaurant,Mobile Phone Shop,Women's Store,Department Store,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop,Dance Studio
4,0,Ajmer,Rajasthan,74.639000,26.469100,Indian Restaurant,Clothing Store,Vegetarian / Vegan Restaurant,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba


## Cities in Cluster 1

In [56]:
cluster1 = final_df[final_df["Cluster label"] == 1].reset_index(drop = True)
cluster1.head()

,Cluster label,City,State or union territory,longitude,latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Baranagar,West Bengal,88.379217,22.653565,ATM,Bakery,Falafel Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop
1,1,Dhanbad,Jharkhand,86.430964,23.795281,ATM,Train Station,Hotel,Dance Studio,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop
2,1,Hospet,Karnataka,76.387230,15.266493,ATM,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop,Dance Studio
3,1,Junagadh,Gujarat,70.459172,21.518559,ATM,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop,Dance Studio
4,1,Kirari Suleman Nagar,Delhi,77.055555,28.700001,ATM,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Discount Store,Diner,Dhaba,Dessert Shop,Dance Studio


In [49]:
#top venue categories are
cluster

,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brewery,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Café,Camera Store,Campground,Candy Store,Capitol Building,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Dhaba,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Grocery Store,Gujarati Restaurant,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Lounge,Market,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nightclub,Optical Shop,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Stadium,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Wine Shop,Women's Store
count,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000
mean,0.164474,0.052632,0.006579,0.006579,0.013158,0.013158,0.006579,0.006579,0.006579,0.006579,0.006579,0.059211,0.006579,0.013158,0.118421,0.032895,0.032895,0.013158,0.026316,0.006579,0.0065

In [47]:
xv = city_venues.group_by('Venue Category').sum()

AttributeError: 'DataFrame' object has no attribute 'group_by'